In [1]:
url = "https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Italy"

In [2]:
import json
import random
from pathlib import Path

import pandas as pd

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())

In [3]:
web_df = pd.read_html(url)

In [4]:
casedata_df = web_df[4]

In [5]:
# casedata_df.columns

In [6]:
italy_slice_df = casedata_df[
    [
        ("Date", "Date"),
        ("Confirmed", "Total"),
        ("Deaths", "Total"),
        ("Active", "ICU"),
        ("Active", "Total"),
        ("Tested", "Total"),
    ]
][:31]

In [7]:
import string
import numpy as np


def remove_punct(field: str) -> float:
    if type(field) == str:
        return float(field.translate(str.maketrans("", "", string.punctuation)))
    else:
        return float(field)


# remove_punct(">8600"), remove_punct(np.nan), remove_punct(90)
italy_slice_df[("Tested", "Total")] = italy_slice_df[("Tested", "Total")].apply(
    remove_punct
)

In [8]:
italy_population = 60782668
india_population = 1326093247
one_million = 10 ** 7
italy_slice_df[("Tested", "TotalPerMillionCapita")] = (
    italy_slice_df[("Tested", "Total")] * one_million / italy_population
)

In [9]:
italy_slice_df.to_csv("ItalyMedicHist.csv", index=False)

In [10]:
# italy_slice_df[("Tested", "TotalPerMillionCapita")]

In [30]:
india_web_df = pd.read_html("https://mohfw.gov.in")
india_state_wise = india_web_df[0]
india_state_wise.drop(columns=["S. No."], inplace=True)

In [32]:
india_state_wise["State"] = india_state_wise["Name of State / UT"]
india_state_wise["Confirmed"] = (
    india_state_wise["Total Confirmed cases (Indian National)"]
    + india_state_wise["Total Confirmed cases ( Foreign National )"]
)
india_state_wise["Deaths"] = india_state_wise["Death"]

In [33]:
india_state_wise["Active"] = (
    india_state_wise["Confirmed"]
    - india_state_wise["Cured/Discharged/Migrated"]
    - india_state_wise["Deaths"]
)

In [34]:
india_state_wise.drop(
    columns=[
        "Name of State / UT",
        "Total Confirmed cases (Indian National)",
        "Total Confirmed cases ( Foreign National )",
        "Cured/Discharged/Migrated",
        "Death"
    ]
)

,State,Confirmed,Deaths,Active
0,Andhra Pradesh,3,0,3
1,Chhattisgarh,1,0,1
2,Delhi,26,1,20
3,Gujarat,7,0,7
4,Haryana,17,0,17
5,Himachal Pradesh,2,0,2
6,Karnataka,15,1,13
7,Kerala,40,0,37
8,Madhya Pradesh,4,0,4
9,Maharashtra,63,1,62


In [35]:
india_slice_df = india_state_wise[["State", "Confirmed", "Active", "Deaths"]][:1]

In [37]:
italy_new_df = pd.DataFrame(italy_slice_df[[("Date","Date"), ("Active", "ICU"), ("Active", "Total")]])

In [38]:
italy_new_df.columns = ["Date", "Active-ICU", "Active-Total"]

In [39]:
def replace_punckt(field:str)->float:
    if field == '–':
        return float(np.nan)
    else: return float(field)
italy_new_df["Active-ICU"] = italy_new_df["Active-ICU"].apply(replace_punckt)

In [40]:
italy_new_df["ICU-to-TotalRatio"] = italy_new_df["Active-ICU"].astype(float) / italy_new_df["Active-Total"].astype(float)

In [41]:
italy_new_df

,Date,Active-ICU,Active-Total,ICU-to-TotalRatio
0,2020-01-31,NaN,2,NaN
1,2020-02-06,NaN,3,NaN
2,2020-02-21,NaN,19,NaN
3,2020-02-22,18.0,75,0.240000
4,2020-02-23,25.0,147,0.170068
5,2020-02-24,27.0,199,0.135678
6,2020-02-25,35.0,309,0.113269
7,2020-02-26,36.0,430,0.083721
8,2020-02-27,56.0,588,0.095238
9,2020-02-28,64.0,821,0.077954
